In [1]:
import pandas as pd
import re
import warnings
import firebase_admin
from firebase_admin import credentials, firestore

import math

warnings.filterwarnings("ignore")

In [2]:
cred = credentials.Certificate("slts-8e29d-firebase-adminsdk-i6d1u-d3ea7654ac.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
df = pd.read_csv("data_fire_18.csv")
df

,Timestamp,Which GROUP does the student belong to?,Student's Full Name,Student's Date of Birth (DOB),Student's Gender,District the student belongs to.,Name of the Samithi.,Student's Year of Joining Balvikas,Student's Date of Joining Balvikas (Optional),Has the student passed Group 2 exam?,...,Number of accompanying Mahilas(Women) who will need accommodation,Any food allergies for student? (Optional),Check In Date,Check In Time,Check out Date,Check out Time,Valid Group Selected based on DOB?,Valid Events Registered?,Overall Registration Status,Remarks
0,11/5/2024 13:45:27,Group 2,SAMYUTHA SRIRAM,"May 3, 2013",Female,Chennai South,Alwarpet,2022,"June 14, 2022",NaN,...,1.0,NaN,"December 25, 2024",6 AM,"December 25, 2024",830 PM,Accepted,Accepted,Accepted,
1,11/7/2024 22:24:42,Group 1,ANIRUDH SAI GUHAN .R,"March 19, 2016",Male,Coimbatore,Podanur,2020,"November 13, 2020",NaN,...,0.0,No,"December 25, 2024",6.30am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
2,11/8/2024 16:07:39,Group 1,G.LAKSHITH,"June 7, 2016",Male,Coimbatore,Saibaba colony,2022,"June 30, 2022",NaN,...,0.0,No,"December 25, 2024",6.30am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
3,11/8/2024 21:10:59,Group 1,SAI CHARANYA SRI P.S,"April 16, 2016",Female,Coimbatore,Podanur,2020,"June 5, 2020",NaN,...,0.0,NaN,"December 25, 2024",6.30 am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
4,11/10/2024 22:42:34,Group 2,S.SAI ESHWAR,"October 24, 2013",Male,Tiruvannamalai,Tiruvannamalai main samithi,2023,"June 18, 2023",NaN,...,1.0,No,"December 24, 2024",11 pm,"December 25, 2024",6:00pm,Accepted,Accepted,Accepted,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,12/12/2024 15:12:44,Group 2,V VEDASHREE,"January 3, 2015",Female,Chennai North West,ASHTALAKSHMI NAGAR,2021,"November 20, 2021",NaN,...,1.0,NO,"December 24, 2024",NaN,"December 25, 2024",NaN,Accepted,Accepted,Accepted,
775,12/12/2024 15:12:45,Group 2,KANISHA B.M.,"January 24, 2013",Female,Chennai North West,GILL NAGAR,2022,"July 9, 2022",NaN,...,2.0,NO,"December 24, 2024",3.00PM,"December 25, 2024",9PM,Accepted,Accepted,Accepted,
776,12/12/2024 15:12:46,Group 2,ROSHAN SARAVANA,"October 7, 2013",Male,Chennai North West,ANNA NAGAR EAST,2019,"November 30, 2019",NaN,...,1.0,NO,"December 24, 2024",3.00PM,"December 25, 2024",9PM,Accepted,Accepted,Accepted,
777,12/12/2024 15:12:47,Group 2,GAYATHRI R S,"March 25, 2014",Female,Chennai North West,SAI SAHARA,2021,"January 24, 2021",NaN,...,NaN,NO,"December 24, 2024",7.00AM,"December 25, 2024",9PM,Accepted,Accepted,Accepted,


In [4]:
def formatModeOfTravel(val):
    val = str(val)
    if "train" in val.lower() or "traom" in val.lower() or "teajn" in val.lower():
        return "Train"
    if "bus" in val.lower():
        return "Bus"
    if "car" in val.lower():
        return "Car"
    if "van" in val.lower():
        return "Van"
    if "flight" in val.lower():
        return "Flight"
    if "own" in val.lower():
        return "Self"
    if "no" in val.lower():
        return "-"
    if "KOVAI" in val:
        return "Train"

df["Mode of Travel"] = df["Mode of Travel"].apply(formatModeOfTravel)
df["Mode of Travel"].value_counts()

Mode of Travel
Train     432
Bus       123
Car        41
Van        33
Self       29
Flight      2
-           1
Name: count, dtype: int64

In [5]:
df["Mode of Travel.1"] = df["Mode of Travel.1"].apply(formatModeOfTravel)
df["Mode of Travel.1"].value_counts()

Mode of Travel.1
Train     379
Bus       155
Car        41
Van        31
Self       30
-           3
Flight      1
Name: count, dtype: int64

In [6]:
# Function to clean and convert time values to HH:MM <AM/PM> format
def standardize_time_format(time_str):
    try:
        # Handle missing or invalid values
        if pd.isnull(time_str) or not isinstance(time_str, str) or not time_str.strip():
            return ''

        # Normalize input by removing extra spaces and converting to lowercase
        time_str = time_str.strip().lower()

        time_str = re.sub(r'[\s,;]+', ':', time_str) 
        time_str = re.sub(r'::+', ':', time_str)      

        # Specific case handling
        if "noon" in time_str:
            return "12:00 PM"
        if "midnight" in time_str:
            return "12:00 AM"
            
        time_str = re.sub(r'^(\d{1,2})(am|pm)$', r'\1:00 \2', time_str) 
        time_str = re.sub(r'^(\d{1,2}):(\d{2})(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2})\s+(am|pm)$', r'\1:00 \2', time_str)  
        time_str = re.sub(r'^(\d{1,2}):(\d{2})\s?(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2}):\s?(am|pm)$', r'\1:00 \2', time_str)  
        time_str = re.sub(r'^(\d{1,2})m$', r'\1:00 am', time_str) 
        time_str = re.sub(r'^(\d{1,2}):(\d{2}):?(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})\s?(am|pm)$', r'\1:\2 \3', time_str) 
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):?(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2})\.(am|pm)$', r'\1:00 \2', time_str)  
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):?a\.m\.?$', r'\1:\2 am', time_str) 
        time_str = re.sub(r'^(\d{1,2})\.:a\.m\.?$', r'\1:00 am', time_str)
        time_str = re.sub(r'^0?(\d{1,2})\.(\d{2})\.?:am$', r'\1:\2 am', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):p\.m$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})(\d{2}):pm$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):p\.m\.?$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.p\.m$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(2[0-3]|[01]?\d)\.(\d{2}):hrs$', lambda m: f"{int(m.group(1)) % 12 or 12}:{m.group(2)} {'pm' if int(m.group(1)) >= 12 else 'am'}", time_str)
        time_str = re.sub(r'^(\d{1,2}):a\.m\.?$', r'\1:00 am', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})mpm$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^0?(\d{1,2})\.?:00:pm$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^nil$', '', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^0?(\d{1,2}):(\d{2})$', lambda m: f"{int(m.group(1)) % 12 or 12}:{m.group(2)} {'pm' if int(m.group(1)) >= 12 else 'am'}", time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})p\.m$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})p\.m$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^0?(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})p\.m$', r'\1:\2 pm', time_str)
 
        match = re.match(r'^(\d{1,2}):(\d{2})\s*(am|pm)$', time_str)
        if not match:
            print(f"Invalid time format: {time_str}")
            return ''

        hour, minute, period = match.groups()
        hour = int(hour)
        minute = int(minute)

        if hour < 1 or hour > 12 or minute < 0 or minute >= 60:
            print(f"Out of range time: {time_str}")
            return ''
        # Format time as HH:MM <AM/PM>
        formatted_time = f"{hour}:{minute:02d} {period.upper()}"
        return formatted_time
    except Exception as e:
        print(f"Error processing {time_str}: {e}")
        return ''

def unifyLength(val):
    val = str(val)
    val = val.strip()
    if len(val) != 0 and len(val) != 8:
        val = "0" + val
    return val

# Columns to process
time_columns = ['Check In Time', 'Check out Time', 'Time of arrival', 'Time of departure']
# Apply the transformation
for col in time_columns:
    if col in df.columns:
        df[col] = df[col].apply(standardize_time_format)
        df[col] = df[col].apply(unifyLength)

Invalid time format: 


In [7]:
# District

# df["District the student belongs to."].unique()

district = [
    "Chennai South",
    "Coimbatore",
    "Tiruvannamalai",
    "Tirunelveli",
    "Kanchipuram North",
    "Thanjavur",
    "Chennai South East",
    "Kanchipuram South",
    "Madurai",
    "Virudhunagar",
    "Trichy",
    "Dharmapuri / Krishnagiri",
    "Cuddalore",
    "Kanyakumari",
    "Chennai West",
    "Salem",
    "Nilgiris",
    "Tiruvallur East",
    "Tuticorin",
    "Tirupur",
    "Chennai North",
    "Sivaganga&Ramnad",
    "Erode",
    "Chennai North West",
    "Puducherry",
    "Vellore",
    "Dindigul",
    "Chennai East Coast",
    "Karur",
    "Theni",
    "Mayiladuthurai",
    "Nagapattinam",
    "Villupuram"
]

district_code = {}

for i in range(len(district)):
    district_code[district[i]] = str(i + 1).zfill(2)

In [8]:
def processData(df):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values(by="Timestamp")
    df = df.reset_index(drop=True)
    df = df.drop(
        [
            "Timestamp",
            "Valid Group Selected based on DOB?",
            "Valid Events Registered?",
            # "Remarks",
        ],
        axis=1,
    )
    column_mapping = {
        "Which GROUP does the student belong to?": "studentGroup",
        "Student's Full Name": "studentFullName",
        "Student's Date of Birth (DOB)": "dateOfBirth",
        "Student's Gender": "gender",
        "District the student belongs to.": "district",
        "Name of the Samithi.": "samithiName",
        "Student's Year of Joining Balvikas": "yearOfJoiningBalvikas",
        "Student's Date of Joining Balvikas (Optional)": "dateOfJoiningBalvikas",
        "Has the student passed Group 2 exam?": "hasPassedGroup2Exam",
        "GROUP 1 - Select 1st event to register for.": "group1Event1",
        "GROUP 1 - Select 2nd event to register for. (Optional)": "group1Event2",
        "GROUP 1 - Select the group event to register for. (Optional)": "group1GroupEvent",
        "Select 1st event to register for.": "event1",
        "Select 2nd event to register for. (Optional)": "event2",
        'Select "Yes" to register for Quiz.': "isRegisteredForQuiz",
        "Date of arrival": "arrivalDate",
        "Time of arrival": "arrivalTime",
        "Does the student need pickup facility.": "needsPickup",
        "Mode of Travel": "modeOfTravel",
        "Pick Up Point": "pickupPoint",
        "Date of departure": "departureDate",
        "Time of departure": "departureTime",
        "Student needs drop facility.": "needsDrop",
        "Mode of Travel.1": "modeOfTravelForDrop",
        "Drop Off Point": "dropOffPoint",
        "Are adults accompanying the student?": "hasAccompanyingAdults",
        "Number of Gents(Men) accompanying student": "numMaleAccompanying",
        "Number of Mahilas(Women) accompanying student": "numFemaleAccompanying",
        "Number of non-participating siblings accompanying the student": "numNonParticipatingSiblings",
        "Name of the accompanying person": "accompanyingPersonName",
        "Gender of the accompanying person": "accompanyingPersonGender",
        "Relation": "accompanyingPersonRelation",
        "Contact Number of accompanying person": "accompanyingPersonContact",
        "Age of accompanying person": "accompanyingPersonAge",
        "Does the student need accommodation?": "needsAccommodation",
        "Number of accompanying Gents(Male) who will need accommodation": "numMaleAccompanyingNeedAccommodation",
        "Number of accompanying Mahilas(Women) who will need accommodation": "numFemaleAccompanyingNeedAccommodation",
        "Any food allergies for student? (Optional)": "foodAllergies",
        "Check In Date": "checkInDate",
        "Check In Time": "checkInTime",
        "Check out Date": "checkOutDate",
        "Check out Time": "checkOutTime",
        "Overall Registration Status": "overallRegistrationStatus",
        "Remarks": "remarks",
    }
    df = df.rename(columns=column_mapping)
    return df

group_code = {
    "Group 1": "G1",
    "Group 2": "G2",
    "Group 3": "G3",
    "General Category": "G4",
}

gender_code = {"Male": "M", "Female": "F"}

def prepareData(df):
    df = processData(df)
    data = df.to_dict(orient="records")
    uid = 1

    # Convert nan fields to empty strings
    for i in range(len(data)):
        for key in data[i].keys():
            if (
                key
                in [
                    "group1Event1",
                    "group1Event2",
                    "event1",
                    "event2",
                    "group1GroupEvent",
                ]
                and type(data[i][key]) == float
                and math.isnan(data[i][key])
            ):
                data[i][key] = ""

    for i in range(len(data)):
        if data[i]["studentGroup"] == "Group 1":
            if len(data[i]["group1Event1"]) > 0:
                data[i]["group1Event1"] = f"G1-{data[i]['group1Event1']}"

            if len(data[i]["group1Event2"]) > 0:
                data[i]["group1Event2"] = f"G1-{data[i]['group1Event2']}"

            data[i]["registeredEvents"] = [
                data[i]["group1Event1"],
                data[i]["group1Event2"],
                data[i]["group1GroupEvent"],
            ]

            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["hasPassedGroup2Exam"]

        elif data[i]["studentGroup"] == "Group 2":
            if "GROUP" not in data[i]["event1"] and len(data[i]["event1"]) > 0:
                data[i]["event1"] = f"G2-{data[i]['event1']}"
            if "GROUP" not in data[i]["event2"] and len(data[i]["event2"]) > 0:
                data[i]["event2"] = f"G2-{data[i]['event2']}"

            data[i]["registeredEvents"] = [data[i]["event1"], data[i]["event2"]]
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["hasPassedGroup2Exam"]

        elif data[i]["studentGroup"] == "Group 3":
            if "GROUP" not in data[i]["event1"] and len(data[i]["event1"]) > 0:
                data[i]["event1"] = f"G3-{data[i]['event1']}"
            if "GROUP" not in data[i]["event2"] and len(data[i]["event2"]) > 0:
                data[i]["event2"] = f"G3-{data[i]['event2']}"

            data[i]["registeredEvents"] = [data[i]["event1"], data[i]["event2"]]
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]

        else:
            data[i]["registeredEvents"] = []
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["hasPassedGroup2Exam"]

        data[i]["registeredEvents"] = [
            x for x in data[i]["registeredEvents"] if str(x) != ""
        ]

        data[i]["totalParticipatingEvents"] = len(data[i]["registeredEvents"])

        data[i][
            "studentId"
        ] = f"{group_code[data[i]['studentGroup']]}{gender_code[data[i]['gender']]}{district_code[data[i]['district']]}{str(len(data[i]['registeredEvents'])).zfill(2)}{str(uid).zfill(3)}"
        uid += 1
        # print(data[i]['studentId'])

    return data

In [9]:
def addDataToFirebase(df):
    data = prepareData(df)

    for i in range(len(data)):
        db.collection("registrationData").document(data[i]["studentId"]).set(data[i])
        # print(f"Done: {data[i]['studentId']}")


def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        # print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

In [10]:
# Delete all documents in the collection
delete_collection(db.collection("registrationData"), 10)

addDataToFirebase(df)